In [13]:
# Initial imports
import os # Miscellaneous operating system interfaces (os.getenv)
import requests # requests.get
import pandas as pd
from dotenv import load_dotenv #Reads the key-value pair from .env file and adds them to environment variable.
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation
from pathlib import Path
import json # lightweight data interchange format 
import matplotlib.pyplot as plt

%matplotlib inline
import FundamentalAnalysis as fa

In [14]:
# Load .env enviroment variables
load_dotenv()
from pathlib import Path
env_path = Path('C:/Users/Khemero/.env')

In [18]:
# Set Alpaca API key and secret
api_key = os.getenv("FA")

In [19]:
ticker = "AAPL"

In [20]:
# Show the available companies
companies = fa.available_companies(api_key)


In [21]:
companies

,name,price,exchange
symbol,,,
SPY,SPDR S&P 500,369.18,NYSE Arca
CMCSA,Comcast Corp,50.90,Nasdaq Global Select
KMI,Kinder Morgan Inc,14.21,New York Stock Exchange
INTC,Intel Corp,47.46,Nasdaq Global Select
MU,Micron Technology Inc,71.46,Nasdaq Global Select
...,...,...,...
WNW,Wunong Net Technology Co Ltd,77.98,NasdaqGS
COOLU,Corner Growth Acquisition Corp.,10.35,NasdaqGS
SCOAU,ScION Tech Growth I,10.18,NasdaqGS


In [24]:
# Collect general company information
profile = fa.profile(ticker, api_key)

# Collect recent company quotes
quotes = fa.quote(ticker, api_key)

# Collect market cap and enterprise value
entreprise_value = fa.enterprise(ticker, api_key)

# Show recommendations of Analysts
ratings = fa.rating(ticker, api_key)

# Obtain DCFs over time
dcf_annually = fa.discounted_cash_flow(ticker, api_key, period="annual")
dcf_quarterly = fa.discounted_cash_flow(ticker, api_key, period="quarter")

# Collect the Balance Sheet statements
balance_sheet_annually = fa.balance_sheet_statement(ticker, api_key, period="annual")
balance_sheet_quarterly = fa.balance_sheet_statement(ticker, api_key, period="quarter")

# Collect the Income Statements
income_statement_annually = fa.income_statement(ticker, api_key, period="annual")
income_statement_quarterly = fa.income_statement(ticker, api_key, period="quarter")

# Collect the Cash Flow Statements
cash_flow_statement_annually = fa.cash_flow_statement(ticker, api_key, period="annual")
cash_flow_statement_quarterly = fa.cash_flow_statement(ticker, api_key, period="quarter")

# Show Key Metrics
key_metrics_annually = fa.key_metrics(ticker, api_key, period="annual")
key_metrics_quarterly = fa.key_metrics(ticker, api_key, period="quarter")

# Show a large set of in-depth ratios
financial_ratios_annually = fa.financial_ratios(ticker, api_key, period="annual")
financial_ratios_quarterly = fa.financial_ratios(ticker, api_key, period="quarter")

# Show the growth of the company
growth_annually = fa.financial_statement_growth(ticker, api_key, period="annual")
growth_quarterly = fa.financial_statement_growth(ticker, api_key, period="quarter")

# Download general stock data
stock_data = fa.stock_data(ticker, period="ytd", interval="1d")

# Download detailed stock data
stock_data_detailed = fa.stock_data_detailed(ticker, api_key, begin="2000-01-01", end="2020-01-01")


In [25]:
profile

,0
symbol,AAPL
price,126.655
beta,1.33758
volAvg,121687500
mktCap,2153362880000
lastDiv,1.4225
range,53.1525-137.98
changes,-2.045
companyName,Apple Inc
currency,USD
